<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=5ef0fe91cc12e166c735b63cfac8c0503cb07507e6c4f2a4f2112c878eda70ea
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.1
    Uninstalling protobuf-6.33.1:
      Successfully uninstalled protobuf-6.33.1


  Attempting uninstall: yfinance
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-11-24 11:06:08
-------------------
qualified stocks: 87
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  400.00
Current:  1.43 C
-------------------
Today PnL: -41.96 K (-0.29%)
Current PnL: -23.06 L (-15.11%)
CY Booked + Current PnL: -9.17 L (-6.01%)
-------------------
Total profit:  1.52 L
Total loss:  -24.58 L
-------------------
Total Booked + Current PnL: 17.81 L (14.18%)
Total Booked PnL: 40.87 L (32.53%)
Curr Year Booked PnL: 13.89 L (9.73%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.31 C
Est FTT PnL: 88.47 L (61.98%)
Deployed:  1.26 C
Current:  1.43 C
CAGR/XIRR %: 7.28%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,TTKPRESTIG,770.00,0.55,-13.76,16.06,0.09,13958.0,-13865.0,86912.0,101.71,52.0,M-SC,4.27,253.0,-0.99,0.61,14.18,OX40N,NTT,DURABLES
39,INDIGOPNTS,1408.00,0.12,-8.88,9.76,0.02,15524.0,-15498.0,159061.0,142.75,71.0,M-SC,5.45,234.0,-1.00,1.11,38.36,OX40N,NTT,PAINTS
43,ITC,452.00,-0.27,-0.62,11.12,10.44,22118.0,-1237.0,198901.0,-40.74,48.0,X-LC,0.76,5.0,-0.06,1.39,4.91,X40,NTT,FMCG
35,ICICIGI,2252.93,0.34,7.29,12.03,20.19,22499.0,12703.0,187023.0,-16.02,52.0,X-MC,7.67,68.0,0.56,1.31,23.50,X40,ATH,INSURANCE
53,NESTLEIND,1377.00,-0.51,12.33,8.07,21.40,24062.0,32737.0,298163.0,4.17,58.0,X-LC,7.92,12.0,1.36,2.09,19.80,XY25,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
26,GILLETTE,11206.78,1.98,-0.27,32.77,32.42,82979.0,-681.0,253215.0,-18.02,38.0,X-SC,6.63,85.0,-0.01,1.77,14.79,X40,ATH,FMCG
7,ATULAUTO,844.00,1.91,-20.99,84.24,45.57,131207.0,-41378.0,155754.0,3370.27,43.0,M-SC,4.00,244.0,-0.32,1.09,10.43,XY24,NTT,AUTO
85,WIPRO,420.00,1.69,1.98,68.93,72.27,119110.0,3350.0,172798.0,-12.43,59.0,M-LC,4.59,99.0,0.03,1.21,8.27,XR,NTT,IT
29,HAPPSTMNDS,1480.71,1.57,-36.99,193.38,84.85,156158.0,-47413.0,80752.0,-25.05,47.0,H-SC,14.26,147.0,-0.30,0.57,3.64,AR,ATH,IT
56,QUESS,424.00,1.53,-28.22,99.00,42.84,46195.0,-18344.0,46662.0,-53.17,39.0,X-SC,37.23,83.0,-0.40,0.33,1.53,XY24,NTT,MISC


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.0,-2.89,-15.28,114.43,81.67,90948.0,-14331.0,79479.0,7144.44,45.0,L-SC,18.45,271.0,-0.16,0.56,53.91,XR,NTT,CERAMICS
81,VALIANTORG,838.0,-2.83,-45.76,221.07,74.15,196178.0,-74865.0,88740.0,-259.70,26.0,H-SC,14.56,149.0,-0.38,0.62,13.75,XR,NTT,CHEMICALS
27,GLAXO,3466.2,-2.53,-2.27,42.37,39.15,82523.0,-4516.0,194768.0,-21.47,28.0,X-MC,9.07,60.0,-0.05,1.36,26.08,X40,ATH,PHARMA
16,CERA,9475.0,-2.44,-28.77,73.92,23.89,92625.0,-50599.0,125304.0,-31.26,25.0,H-SC,3.97,157.0,-0.55,0.88,8.22,OX40N,NTT,CERAMICS
58,RAJOOENG,143.1,-2.29,-29.22,97.24,39.61,70548.0,-29950.0,72550.0,-52.19,21.0,H-SC,19.25,135.0,-0.42,0.51,0.00,AR,ATH,MISC


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
38,INDIAMART,4810.62,-0.82,-0.19,107.11,106.72,131856.0,-233.0,123103.0,-50.63,34.0,H-SC,4.09,139.0,-0.0,0.86,23.15,AR,ATH,MISC


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-0.76,-4.71,118.19,107.91,166280.0,-6955.0,140689.0,-22.58,34.0,M-SC,10.34,216.0,-0.04,0.99,0.00,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,0.12,-8.88,9.76,0.02,15524.0,-15498.0,159061.0,142.75,71.0,M-SC,5.45,234.0,-1.00,1.11,38.36,OX40N,NTT,PAINTS
78,TTKPRESTIG,770.00,0.55,-13.76,16.06,0.09,13958.0,-13865.0,86912.0,101.71,52.0,M-SC,4.27,253.0,-0.99,0.61,14.18,OX40N,NTT,DURABLES
47,KANSAINER,340.00,-0.31,-22.54,46.50,13.47,97125.0,-60795.0,208872.0,-68.12,27.0,H-SC,1.52,158.0,-0.63,1.46,6.21,XY24,NTT,PAINTS
73,TATAELXSI,9161.00,0.24,-23.13,71.76,32.03,72719.0,-30499.0,101336.0,-16.88,48.0,H-SC,6.65,148.0,-0.42,0.71,13.51,OX40N,NTT,IT


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.0,1.69,1.98,68.93,72.27,119110.0,3350.0,172798.0,-12.43,59.0,M-LC,4.59,99.0,0.03,1.21,8.27,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.00,1.69,1.98,68.93,72.27,119110.0,3350.0,172798.0,-12.43,59.0,M-LC,4.59,99.0,0.03,1.21,8.27,XR,NTT,IT
38,INDIAMART,4810.62,-0.82,-0.19,107.11,106.72,131856.0,-233.0,123103.0,-50.63,34.0,H-SC,4.09,139.0,-0.00,0.86,23.15,AR,ATH,MISC
25,FINCABLES,1641.55,-0.76,-4.71,118.19,107.91,166280.0,-6955.0,140689.0,-22.58,34.0,M-SC,10.34,216.0,-0.04,0.99,0.00,OX40N,ATH,CABLES
86,ZYDUSLIFE,1286.17,0.37,-2.17,38.64,35.63,79221.0,-4552.0,205022.0,-15.85,34.0,H-MC,4.52,119.0,-0.06,1.44,14.14,AR,ATH,PHARMA
37,IEX,219.00,0.48,-3.77,54.44,48.63,104755.0,-7531.0,192423.0,-36.00,54.0,H-SC,15.57,136.0,-0.07,1.35,7.35,XR,NTT,MISC


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
46,JSWINFRA,342.00,-1.39,-10.33,28.50,15.23,51201.0,-20696.0,179651.0,-23.19,22.0,X-MC,6.52,66.0,-0.40,1.26,18.59,X40N,NTT,REALTY
76,TMPV,600.00,-1.27,-15.25,67.76,42.18,158735.0,-42149.0,234261.0,-24.30,22.0,X-LC,0.43,3.0,-0.27,1.64,0.00,XY24,NTT,AUTO
34,HONAUT,58357.33,0.45,-15.79,70.19,43.32,96273.0,-25712.0,137160.0,-27.69,26.0,X-SC,2.03,90.0,-0.27,0.96,5.31,X40N,ATH,ELECTRICAL
11,BATAINDIA,2096.00,0.76,-34.98,107.94,35.20,90307.0,-45006.0,83664.0,9.75,26.0,X-SC,12.60,92.0,-0.50,0.59,0.76,X40,NTT,FOOTWEAR
27,GLAXO,3466.20,-2.53,-2.27,42.37,39.15,82523.0,-4516.0,194768.0,-21.47,28.0,X-MC,9.07,60.0,-0.05,1.36,26.08,X40,ATH,PHARMA


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TMPV,600.0,-1.27,-15.25,67.76,42.18,158735.0,-42149.0,234261.0,-24.30,22.0,X-LC,0.43,3.0,-0.27,1.64,0.00,XY24,NTT,AUTO
43,ITC,452.0,-0.27,-0.62,11.12,10.44,22118.0,-1237.0,198901.0,-40.74,48.0,X-LC,0.76,5.0,-0.06,1.39,4.91,X40,NTT,FMCG
36,ICICIPRULI,790.0,0.07,1.34,29.23,30.97,52354.0,2373.0,179111.0,-22.52,49.0,X-MC,1.54,75.0,0.05,1.25,14.07,X40,ATH,INSURANCE
3,ACC,3906.0,-0.28,-23.30,114.03,64.15,208105.0,-55451.0,182500.0,-55.00,42.0,X-SC,1.89,82.0,-0.27,1.28,2.37,XY24,BTT,CEMENT
66,SIEMENS,4671.5,-0.46,-15.13,47.88,25.51,75624.0,-28150.0,157945.0,-3.08,52.0,H-LC,1.92,49.0,-0.37,1.11,16.39,AR,ATH,ELECTRICAL


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
9,BAJAJHFL,181.50,-0.19,-13.85,72.32,48.45,124926.0,-27765.0,172741.0,-20.09,28.0,X-MC,7.48,64.0,-0.22,1.21,0.00,X40N,ATH,FINANCE
76,TMPV,600.00,-1.27,-15.25,67.76,42.18,158735.0,-42149.0,234261.0,-24.30,22.0,X-LC,0.43,3.0,-0.27,1.64,0.00,XY24,NTT,AUTO
11,BATAINDIA,2096.00,0.76,-34.98,107.94,35.20,90307.0,-45006.0,83664.0,9.75,26.0,X-SC,12.60,92.0,-0.50,0.59,0.76,X40,NTT,FOOTWEAR
54,PAGEIND,51605.08,0.04,-4.97,32.66,26.07,50819.0,-8140.0,155600.0,-30.06,36.0,X-MC,8.86,55.0,-0.16,1.09,0.87,X40,ATH,APPARELS
56,QUESS,424.00,1.53,-28.22,99.00,42.84,46195.0,-18344.0,46662.0,-53.17,39.0,X-SC,37.23,83.0,-0.40,0.33,1.53,XY24,NTT,MISC


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,QUESS,424.00,1.53,-28.22,99.00,42.84,46195.0,-18344.0,46662.0,-53.17,39.0,X-SC,37.23,83.0,-0.40,0.33,1.53,XY24,NTT,MISC
60,RELAXO,1176.00,-0.71,-47.48,189.98,52.31,144846.0,-68917.0,76243.0,-44.05,28.0,X-SC,3.78,91.0,-0.48,0.53,2.06,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,0.76,-34.98,107.94,35.20,90307.0,-45006.0,83664.0,9.75,26.0,X-SC,12.60,92.0,-0.50,0.59,0.76,X40,NTT,FOOTWEAR
52,MEDANTA,1486.00,0.01,2.45,21.11,24.08,27456.0,3112.0,130062.0,-5.81,44.0,X-SC,4.75,89.0,0.11,0.91,22.55,XY24,NTT,HEALTHCARE
22,DIXON,18931.72,-1.10,-4.95,27.92,21.58,37189.0,-6939.0,133200.0,-52.09,33.0,X-MC,7.02,56.0,-0.19,0.93,17.92,X40N,ATH,IT


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4389.97,0.04,-14.36,39.28,19.28,116374.0,-49687.0,296269.0,-25.24,64.0,X-LC,7.64,1.0,-0.43,2.08,9.53,X40,ATH,IT
41,INFY,2275.00,0.55,5.18,47.25,54.87,155493.0,16197.0,329085.0,-18.16,60.0,X-LC,3.31,2.0,0.10,2.31,13.89,X40,BTT,IT
76,TMPV,600.00,-1.27,-15.25,67.76,42.18,158735.0,-42149.0,234261.0,-24.30,22.0,X-LC,0.43,3.0,-0.27,1.64,0.00,XY24,NTT,AUTO
82,VBL,671.64,0.11,-9.47,49.87,35.67,142588.0,-29922.0,285920.0,-20.30,41.0,X-LC,6.05,4.0,-0.21,2.00,3.07,X40N,ATH,FMCG
43,ITC,452.00,-0.27,-0.62,11.12,10.44,22118.0,-1237.0,198901.0,-40.74,48.0,X-LC,0.76,5.0,-0.06,1.39,4.91,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,-2.89,-15.28,114.43,81.67,90948.0,-14331.0,79479.0,7144.44,45.0,L-SC,18.45,271.0,-0.16,0.56,53.91,XR,NTT,CERAMICS
68,SONACOMS,806.63,1.52,-11.63,57.92,39.54,51775.0,-11769.0,89390.0,-30.90,69.0,M-SC,6.86,220.0,-0.23,0.63,26.05,AR,ATH,AUTO
51,MASFIN,398.61,-2.05,-4.91,28.36,22.05,26422.0,-4815.0,93165.0,-17.80,51.0,H-SC,7.22,164.0,-0.18,0.65,35.42,XR,ATH,FINANCE
70,SURYODAY,216.00,-0.77,-20.28,54.33,23.03,77561.0,-36312.0,142759.0,55.70,47.0,H-SC,11.07,167.0,-0.47,1.00,41.65,XR,NTT,BANKS
39,INDIGOPNTS,1408.00,0.12,-8.88,9.76,0.02,15524.0,-15498.0,159061.0,142.75,71.0,M-SC,5.45,234.0,-1.00,1.11,38.36,OX40N,NTT,PAINTS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,0.12,-8.88,9.76,0.02,15524.0,-15498.0,159061.0,142.75,71.0,M-SC,5.45,234.0,-1.00,1.11,38.36,OX40N,NTT,PAINTS
5,ASIANPAINT,3460.25,0.03,0.58,20.25,20.95,51279.0,1461.0,253229.0,-0.49,76.0,X-LC,12.74,31.0,0.03,1.77,36.42,X40,ATH,PAINTS
40,INDUSINDBK,1800.00,0.41,-37.25,111.76,32.89,54148.0,-28756.0,48450.0,-698.49,65.0,L-MC,5.62,259.0,-0.53,0.34,33.43,XR,NTT,BANKS
50,LTIM,7201.88,1.25,9.69,20.03,31.66,54081.0,23849.0,270000.0,5.43,65.0,H-LC,12.55,43.0,0.44,1.89,49.81,X200,ATH,IT
0,5PAISA,593.00,-2.10,-36.09,76.67,12.90,107054.0,-78866.0,139630.0,105.53,56.0,H-SC,14.57,161.0,-0.74,0.98,16.10,OX40N,NTT,FINANCE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.78
1,25,44.66
2,50,75.44


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.57
MC    29.24
LC    25.18
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     27.88
X40      21.22
X40N     11.82
XR        9.64
XY25      9.10
AR        8.99
OX40N     7.68
X200      1.89
SR        0.99
MH        0.78
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    23.93
X-MC    22.82
X-LC    20.00
M-SC    12.07
X-SC     8.18
H-MC     4.71
H-LC     3.00
L-SC     1.39
M-MC     1.37
M-LC     1.21
L-LC     0.97
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.82,-4.75,38.63
IT,13.87,-14.34,73.36
FINANCE,9.41,-15.36,64.11
MISC,7.41,-22.69,72.95
PAINTS,5.99,-7.73,23.73
ELECTRICAL,5.89,-11.75,52.69
INSURANCE,4.40,-0.51,35.93
PHARMA,3.83,-2.11,34.79
AUTO,3.36,-19.88,71.28


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3055991.0,21
AR,1291057.0,10
XR,1288411.0,13
X40,1008441.0,14
X40N,758544.0,9
OX40N,695840.0,10
XY25,342309.0,6
SR,279173.0,2
MH,73219.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3565592.0,25
M-SC,1336615.0,15
X-MC,1222028.0,16
X-LC,907453.0,11
X-SC,751435.0,8
H-MC,407312.0,3
L-SC,263509.0,3
H-LC,129705.0,2
M-LC,119110.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1222739.0      6
           AR         876202.0      5
           XR         790388.0      7
M-SC       XY24       781473.0      6
X-MC       X40        443730.0      7
X-LC       X40        391425.0      5
X-MC       XY24       348830.0      3
H-SC       OX40N      323871.0      4
X-SC       X40N       296393.0      3
M-SC       OX40N      283545.0      5
X-SC       XY24       281756.0      3
H-SC       SR         279173.0      2
X-MC       X40N       250222.0      4
X-LC       XY24       231195.0      2
H-MC       AR         217314.0      2
X-LC       X40N       211929.0      2
H-MC       XY24       189998.0      1
X-MC       XY25       179246.0      2
L-SC       XR         175085.0      2
X-SC       X40        173286.0      2
M-SC       XR         149680.0      2
           AR         121917.0      2
M-LC       XR         119110.0      1
L-SC       OX40N       88424.0      1
H-LC       AR          75624.0      1
H-SC       MH          73219.0      1
X-LC       XY25        72904.0      2
L-MC       XR          54148.0      1
H-LC       X200        54081.0      1
M-MC       XY25        49639.0      1
L-LC       XY25        40520.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 47.0 seconds
